In [2]:
cd D:\Saarbrucken\EDA_Research\vae-disentanglement\disentanglement_lib_pl

D:\Saarbrucken\EDA_Research\vae-disentanglement\disentanglement_lib_pl


In [12]:
import torch
from torch import nn
import numpy as np
from architectures import encoders, decoders
from common.ops import Flatten3D, Unsqueeze3D, Reshape

In [3]:
enc = getattr(encoders, 'SimpleGaussianConv64CommAss')(10,3,64)
dec = getattr(decoders, 'SimpleConv64CommAss')(10,3,64)

In [4]:
img = torch.rand(size=[1,3,64,64])

In [11]:
enc = nn.Sequential(
    nn.Conv2d(1, 5, 2, 1, 0), # B,5,2,2
    nn.ReLU(True),
    Flatten3D(), # B,5x2x2
    nn.Linear(20, 10),
    nn.Linear(10, num_latent)
)

dec = nn.Sequential(
    Unsqueeze3D(),
    nn.Conv2d(num_latent, 20, 1, 1), 
    nn.ReLU(True),
    Reshape([5, 2, 2]),
    nn.ConvTranspose2d(5, 1, 2, 1, 0)
)



torch.Size([1, 256, 1, 1])


In [31]:
o = Unsqueeze3D()(o)
#print(o.shape, o)
o = nn.Conv2d(2, 20, 1, 1)(o)
o = nn.ReLU(True)(o)
#print(o.shape, o)
o = Reshape([5,2, 2])(o)
#print(o.shape, o)
o = nn.ConvTranspose2d(5, 1, 2, 1, 0)(o)
print(o.shape, o)

torch.Size([1, 1, 3, 3]) tensor([[[[-0.1105, -0.2154, -0.3674],
          [-0.1536, -0.5432, -0.3171],
          [-0.3573, -0.3726, -0.3949]]]],
       grad_fn=<SlowConvTranspose2DBackward>)


# Multiscale + GNN structure test 

In [1]:
cd D:\Saarbrucken\EDA_Research\vae-disentanglement\disentanglement_lib_pl

D:\Saarbrucken\EDA_Research\vae-disentanglement\disentanglement_lib_pl


In [79]:
import torch
import torch.nn as nn
from common.ops import Flatten3D, Unsqueeze3D, Reshape

class MultiScaleEncoder(nn.Module):
    """
    Encoder as used in 'Challenging Common Assumptions in the Unsupervised Learning of Disentangled Representations'
    """
    def __init__(self, feature_dim, in_channels, num_nodes):
        super().__init__()

        self.feature_dim = feature_dim
        self.num_nodes = num_nodes
        self.features_to_take = self.feature_dim // self.num_nodes
        self.batch_size = None

        # in / out feature maps at each scale
        self.scale_3_in, self.scale_3_out = in_channels, 32
        self.scale_2_in, self.scale_2_out = 32, 32
        self.scale_1_in, self.scale_1_out = 32, 64

        # coarsest scale - outputs maps of shape B, 
        self.scale_3 = nn.Sequential(
            nn.Conv2d(self.scale_3_in, self.scale_3_out, 4, 2), # B, 32, 31 x 31
            nn.ReLU(True)
        )
        self.scale_3_feats = nn.Sequential(
            Flatten3D(),
            nn.Linear(self.scale_3_out * 31 * 31, self.feature_dim),
            nn.Tanh()
        )
        # mid scale - outputs maps of shape B, 
        self.scale_2 = nn.Sequential(
            nn.Conv2d(self.scale_2_in, self.scale_2_out, 4, 2), # B, 32, 14 x 14
            nn.ReLU(True)
        )
        self.scale_2_feats = nn.Sequential(
            Flatten3D(),
            nn.Linear(self.scale_2_out * 14 * 14, self.feature_dim),
            nn.Tanh()
        )
        
        # finest scale - outs maps of shape B,
        self.scale_1 = nn.Sequential(
            nn.Conv2d(self.scale_1_in, self.scale_1_out, 4, 2), # B, 64, 6 x 6
            nn.ReLU(True)
        )
        self.scale_1_feats = nn.Sequential(
            Flatten3D(),
            nn.Linear(self.scale_1_out * 6 * 6, self.feature_dim),
            nn.Tanh()
        )

    def forward(self, x):
        
        self.batch_size = x.shape[0]
        
        scale_3_x = self.scale_3(x)
        scale_3_feats = self.scale_3_feats(scale_3_x)

        scale_2_x = self.scale_2(scale_3_x)
        scale_2_feats = self.scale_2_feats(scale_2_x)

        scale_1_x = self.scale_1(scale_2_x)
        scale_1_feats = self.scale_1_feats(scale_1_x)
        #print(scale_3_feats.shape)
        #print(scale_2_feats.shape)
        #print(scale_1_feats.shape)
        
        # Just stacking gives the shape (3, batch_size, feature_dim). Hence, we need to permute tp get 
        # (batch_size, feature_dim, 3)
        multi_scale_feats = torch.stack([scale_3_feats, scale_2_feats, scale_1_feats]).permute(1,2,0)
        
        multi_scale_feats = multi_scale_feats.reshape(self.batch_size, self.num_nodes, 3 * self.features_to_take )
        
        # reshape like this so that they can be associated with each latent node
        return multi_scale_feats
    
class GCNLayer(nn.Module):
    def __init__(self, c_in, c_out):
        super().__init__()
        self.projection = nn.Linear(c_in, c_out)

    def forward(self, node_feats, adj_matrix):
        """
        Args:
            node_feats: Tensor with node features of shape [batch_size, num_nodes, c_in]
            adj_matrix: Batch of adjacency matrices of the graph. If there is an edge from i to j,
                         adj_matrix[b,i,j]=1 else 0. Supports directed edges by non-symmetric matrices.
                         Assumes to already have added the identity connections.
                         Shape: [batch_size, num_nodes, num_nodes]
        """
        # Num neighbours = number of incoming edges
        num_neighbours = adj_matrix.sum(dim=-1, keepdims=True)
        # prep. msg
        node_feats = self.projection(node_feats)
        #print(node_feats.shape)
        # send msg.
        node_feats = torch.matmul(adj_matrix, node_feats)
        node_feats = node_feats / num_neighbours
        return node_feats

In [80]:
ms_enc = MultiScaleEncoder(4*2, 1, 4)

In [81]:
out = ms_enc(torch.randn(2,1,64,64))
print(out)
print(out.shape)

tensor([[[ 0.3083, -0.1329,  0.0334,  0.2024,  0.0683,  0.0298],
         [-0.1563,  0.0509,  0.0678,  0.1451, -0.1596,  0.0435],
         [-0.1692, -0.0438, -0.0202, -0.1336,  0.1160, -0.0684],
         [ 0.0177,  0.0900, -0.0045, -0.1646,  0.1573,  0.0504]],

        [[ 0.2070,  0.0479,  0.0158, -0.0309,  0.0621,  0.0077],
         [ 0.2464,  0.0283,  0.0510, -0.3034,  0.0586,  0.0516],
         [-0.1682,  0.0111, -0.0081,  0.2839, -0.0977, -0.0650],
         [-0.0549,  0.0296,  0.0189,  0.1485,  0.0469,  0.0219]]],
       grad_fn=<UnsafeViewBackward>)
torch.Size([2, 4, 6])


In [83]:
gcn_layer = GCNLayer(c_in=6,c_out=2)
#gcn_layer.projection.weight.data = torch.eye(6)
#gcn_layer.projection.bias.data = torch.zeros(6)

adj_mat = torch.Tensor([
    [1., 0., 1., 1.],
    [0., 1., 0., 1.],
    [0., 0., 1., 0.],
    [0., 0., 0., 1.]
])

gcn_layer(out, adj_mat)

torch.Size([2, 4, 2])


tensor([[[-0.0474, -0.3366],
         [-0.0742, -0.2990],
         [-0.1257, -0.3129],
         [-0.1129, -0.2830]],

        [[-0.0147, -0.3387],
         [-0.0268, -0.2952],
         [-0.0153, -0.3666],
         [-0.0542, -0.3530]]], grad_fn=<DivBackward0>)